#### Description
This Notebook contains CNN model for cogntive load classification. The dataset used contains $44000$ images from four levels of cognitive load $11000$ from each. Images as spectral temporal representation of EEG data recorded from subjects (11 people) taking a working memory test.

I implemented a simple CNN model to cogntive load classification. The model was designed using Keras library which used tensorflow as backend.

#####  Step 1: Loading necessary libraries

In [1]:
from tensorflow import keras
# import keras
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import load_model
from sklearn.utils import class_weight

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from collections import Counter
import numpy as np
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.multiclass import unique_labels


### Step: Dataset loading and preparation
Images used are 224 by 224 RGB images. I used a Data genaerator python library to load images. Data generator is useful because it load images batch bay batch during training which prevents from running out memory when we load entire dataset at once. The generator also perfoms data augmentation by modifying images by simple rotation, scaling,

In [2]:

num_classes = 4
img_rows, img_cols = 224,224
batch_size =32

# Data path
train_data_dir ="/media/kashraf/Elements/Journal_2022/Datasets/topos_GAN+REAL/stack/cropped_train"
validation_data_dir ="/media/kashraf/Elements/Journal_2022/Datasets/topos_GAN+REAL/stack/cropped_test"

# I used images generator library to load data from path and perform some data augmentation 
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()
 
# This will load training images
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)
# This will load training images
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
nb_train_samples = 30800
nb_validation_samples = 13200


Found 30800 images belonging to 4 classes.
Found 13200 images belonging to 4 classes.


In [3]:
# class_weights = class_weight.compute_class_weight(
#                'balanced',
#                 np.unique(train_generator.classes), 
#                 train_generator.classes)
# print(class_weights)

## Let's create our LittleVGG Model

In [4]:
model = Sequential()

# First CONV-ReLU Layer
model.add(Conv2D(32, (7, 7), padding = 'same', input_shape = (img_rows, img_cols,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second CONV-ReLU Layer
model.add(Conv2D(64, (5, 5), padding = "same"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.4))

model.add(Conv2D(96, (3, 3), padding="same"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3), padding="same",kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding="same",kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding="same",kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.4))


# Final Dense Layer
model.add(Dense(num_classes,activation='softmax',name='hafizzo'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      4736      
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 224, 224, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      51264     
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 112, 112, 64)      0

### Training our LittleVGG Model!

In [5]:
             
checkpoint = ModelCheckpoint("/home/kashraf/Journal_work_Fall2021/modelling/real_gan_models/model_weights_april/cnn_stack_22_jun28-v1.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 7,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 3,
                              verbose = 1,
                              min_delta = 0.000001)

# we put our call backs into a callback list
callbacks = [ checkpoint,reduce_lr]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(learning_rate=0.0001),
              metrics = ['accuracy'])


epochs = 250

history= model.fit(   
    
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/250
962/962 [==============================] - ETA: 0s - loss: 5.6360 - accuracy: 0.4815
Epoch 00001: val_loss improved from inf to 21.84087, saving model to /home/kashraf/Journal_work_Fall2021/modelling/real_gan_models/model_weights_april/cnn_stack_22_jun28-v1.h5
962/962 [==============================] - 74s 77ms/step - loss: 5.6360 - accuracy: 0.4815 - val_loss: 21.8409 - val_accuracy: 0.2503 - lr: 1.0000e-04
Epoch 2/250
962/962 [==============================] - ETA: 0s - loss: 3.3057 - accuracy: 0.7020
Epoch 00002: val_loss improved from 21.84087 to 6.50935, saving model to /home/kashraf/Journal_work_Fall2021/modelling/real_gan_models/model_weights_april/cnn_stack_22_jun28-v1.h5
962/962 [==============================] - 73s 76ms/step - loss: 3.3057 - accuracy: 0.7020 - val_loss: 6.5094 - val_accuracy: 0.2732 - lr: 1.0000e-04
Epoch 3/250
962/962 [==============================] - ETA: 0s - loss: 1.9464 - accuracy: 0.7649
Epoch 00003: val_loss did not improve from 6.50935
9

Epoch 24/250
962/962 [==============================] - ETA: 0s - loss: 0.1256 - accuracy: 0.9963
Epoch 00024: val_loss did not improve from 0.31971
962/962 [==============================] - 73s 76ms/step - loss: 0.1256 - accuracy: 0.9963 - val_loss: 0.3216 - val_accuracy: 0.9413 - lr: 8.0000e-07
Epoch 25/250
962/962 [==============================] - ETA: 0s - loss: 0.1249 - accuracy: 0.9962
Epoch 00025: val_loss did not improve from 0.31971

Epoch 00025: ReduceLROnPlateau reducing learning rate to 1.600000018697756e-07.
962/962 [==============================] - 73s 76ms/step - loss: 0.1249 - accuracy: 0.9962 - val_loss: 0.3209 - val_accuracy: 0.9420 - lr: 8.0000e-07
Epoch 26/250
961/962 [============================>.] - ETA: 0s - loss: 0.1233 - accuracy: 0.9969
Epoch 00026: val_loss did not improve from 0.31971
962/962 [==============================] - 73s 76ms/step - loss: 0.1233 - accuracy: 0.9969 - val_loss: 0.3203 - val_accuracy: 0.9417 - lr: 1.6000e-07
Epoch 27/250
962/962 [

Epoch 74/250
962/962 [==============================] - ETA: 0s - loss: 0.1226 - accuracy: 0.9971
Epoch 00074: val_loss did not improve from 0.31971
962/962 [==============================] - 72s 75ms/step - loss: 0.1226 - accuracy: 0.9971 - val_loss: 0.3206 - val_accuracy: 0.9417 - lr: 1.0486e-18
Epoch 75/250
962/962 [==============================] - ETA: 0s - loss: 0.1229 - accuracy: 0.9972
Epoch 00075: val_loss did not improve from 0.31971
962/962 [==============================] - 72s 75ms/step - loss: 0.1229 - accuracy: 0.9972 - val_loss: 0.3205 - val_accuracy: 0.9419 - lr: 1.0486e-18
Epoch 76/250
962/962 [==============================] - ETA: 0s - loss: 0.1227 - accuracy: 0.9972
Epoch 00076: val_loss did not improve from 0.31971

Epoch 00076: ReduceLROnPlateau reducing learning rate to 2.0971520409814568e-19.
962/962 [==============================] - 72s 75ms/step - loss: 0.1227 - accuracy: 0.9972 - val_loss: 0.3206 - val_accuracy: 0.9420 - lr: 1.0486e-18
Epoch 77/250
962/962 

962/962 [==============================] - ETA: 0s - loss: 0.1221 - accuracy: 0.9975
Epoch 00124: val_loss did not improve from 0.31971

Epoch 00124: ReduceLROnPlateau reducing learning rate to 1.3743896500774647e-30.
962/962 [==============================] - 73s 76ms/step - loss: 0.1221 - accuracy: 0.9975 - val_loss: 0.3204 - val_accuracy: 0.9421 - lr: 6.8719e-30
Epoch 125/250
962/962 [==============================] - ETA: 0s - loss: 0.1224 - accuracy: 0.9972
Epoch 00125: val_loss did not improve from 0.31971
962/962 [==============================] - 73s 76ms/step - loss: 0.1224 - accuracy: 0.9972 - val_loss: 0.3206 - val_accuracy: 0.9418 - lr: 1.3744e-30
Epoch 126/250
962/962 [==============================] - ETA: 0s - loss: 0.1227 - accuracy: 0.9969
Epoch 00126: val_loss did not improve from 0.31971
962/962 [==============================] - 73s 76ms/step - loss: 0.1227 - accuracy: 0.9969 - val_loss: 0.3204 - val_accuracy: 0.9417 - lr: 1.3744e-30
Epoch 127/250
962/962 [=========

Epoch 149/250
962/962 [==============================] - ETA: 0s - loss: 0.1225 - accuracy: 0.9973
Epoch 00149: val_loss did not improve from 0.31971
962/962 [==============================] - 73s 76ms/step - loss: 0.1225 - accuracy: 0.9973 - val_loss: 0.3208 - val_accuracy: 0.9418 - lr: 3.5184e-36
Epoch 150/250
962/962 [==============================] - ETA: 0s - loss: 0.1220 - accuracy: 0.9975
Epoch 00150: val_loss did not improve from 0.31971
962/962 [==============================] - 73s 76ms/step - loss: 0.1220 - accuracy: 0.9975 - val_loss: 0.3208 - val_accuracy: 0.9420 - lr: 3.5184e-36
Epoch 151/250
962/962 [==============================] - ETA: 0s - loss: 0.1225 - accuracy: 0.9972
Epoch 00151: val_loss did not improve from 0.31971

Epoch 00151: ReduceLROnPlateau reducing learning rate to 7.036875521556106e-37.
962/962 [==============================] - 73s 76ms/step - loss: 0.1225 - accuracy: 0.9972 - val_loss: 0.3208 - val_accuracy: 0.9418 - lr: 3.5184e-36
Epoch 152/250
962/9

Epoch 200/250
962/962 [==============================] - ETA: 0s - loss: 0.1225 - accuracy: 0.9973
Epoch 00200: val_loss did not improve from 0.31971
962/962 [==============================] - 73s 75ms/step - loss: 0.1225 - accuracy: 0.9973 - val_loss: 0.3206 - val_accuracy: 0.9419 - lr: 0.0000e+00
Epoch 201/250
962/962 [==============================] - ETA: 0s - loss: 0.1230 - accuracy: 0.9966
Epoch 00201: val_loss did not improve from 0.31971
962/962 [==============================] - 73s 76ms/step - loss: 0.1230 - accuracy: 0.9966 - val_loss: 0.3203 - val_accuracy: 0.9417 - lr: 0.0000e+00
Epoch 202/250
962/962 [==============================] - ETA: 0s - loss: 0.1231 - accuracy: 0.9969
Epoch 00202: val_loss did not improve from 0.31971
962/962 [==============================] - 72s 75ms/step - loss: 0.1231 - accuracy: 0.9969 - val_loss: 0.3206 - val_accuracy: 0.9420 - lr: 0.0000e+00
Epoch 203/250
962/962 [==============================] - ETA: 0s - loss: 0.1226 - accuracy: 0.9970
E

In [6]:
model.save("/home/kashraf/Research_2021/saved_models/cnn_stack_jun28-v1.h5")

OSError: Unable to create file (unable to open file: name = '/home/kashraf/Research_2021/saved_models/cnn_stack_jun28-v1.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)


## Performance Analysis

In [1]:
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# We need to recreate our validation generator with shuffle = false
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
nb_train_samples = 29328
nb_validation_samples = 14672


#Confution Matrix and Classification Report
Y_pred = model.predict_generator(validation_generator, nb_validation_samples // batch_size)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
cnf_matrix = confusion_matrix(validation_generator.classes, y_pred)

plt.imshow(cnf_matrix, interpolation='nearest')
plt.colorbar()
tick_marks = np.arange(len(classes))
_ = plt.xticks(tick_marks, classes, rotation=90)
_ = plt.yticks(tick_marks, classes)

class_names=np.asarray(['CL-2','CL-4','CL-6','CL-8'])
# Plot non-normalized confusion matrix
plot_confusion_matrix(validation_generator.classes, y_pred, classes=class_names,title='',name='conf_matrix')
# Plot normalized confusion matrix
plot_confusion_matrix(validation_generator.classes, y_pred, classes=class_names, normalize=True,title='',name='conf_matrix_Nor')
plt.show()


NameError: name 'validation_datagen' is not defined

### Plotting our Accuracy and Loss Charts

In [ ]:

print(history.history.keys())
# Plotting our loss charts
import matplotlib.pyplot as plt

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_loss_values, label='Validation/Test Loss')
line2 = plt.plot(epochs, loss_values, label='Training Loss')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.savefig('Loss_stack.png', dpi = 1200)
plt.show()

In [ ]:
# Plotting our accuracy charts
import matplotlib.pyplot as plt

history_dict = history.history
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(loss_values) + 1)
line1 = plt.plot(epochs, val_acc_values, label='Validation/Test Accuracy')
line2 = plt.plot(epochs, acc_values, label='Training Accuracy')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.savefig('Accuracy_stack.png', dpi = 1200)
plt.show()

### Evaluation on validation set

In [ ]:
import seaborn as sr
validation_path=validation_data_dir ="/home/kashraf/felix_hd/data_gen_may_2021/Audio_topomaps_June21/stack/validation/"
validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
nb_train_samples = 29328
nb_validation_samples = 6192


#Confution Matrix and Classification Report
Y_pred = model.predict(validation_generator, nb_validation_samples // batch_size)
y_pred = np.argmax(Y_pred, axis=1)


In [ ]:
import pandas as pd
conf_mat=confusion_matrix(validation_generator.classes,y_pred,normalize='true')

conf_df=pd.DataFrame(conf_mat, index=["CL-1","CL-2","CL-3","CL-4"], columns=["CL-1","CL-2","CL-3","CL-4"])
# print("\nFace  accuracy =",accuracy)
# print("\n Face recognition report: \n",report)
fig=plt.figure(figsize=(8,6))
sr.heatmap(conf_df,annot=True,cmap="Blues")
# plt.title("Confusion matrix")
plt.xlabel("PREDICTED LABEL")
plt.ylabel("TRUE LABEL")
plt.savefig("Conf_mat_stack_valid")

##### ROC and AUC

In [ ]:
model_stack=load_model("saved_models/cnn_stack_jun28-v1.h5")
y_pred = model_stack.predict_proba(validation_generator, nb_validation_samples // batch_size)
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
from sklearn.metrics import roc_curve, auc,roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.preprocessing import label_binarize
from  tensorflow.keras.utils import to_categorical 

Y_test=to_categorical(validation_generator.classes)
Y_pred=to_categorical(y_pred)
rocauc=roc_auc_score(Y_test,Y_pred,multi_class="ov")

In [ ]:
rocauc

In [ ]:
# Plot linewidth.
from itertools import cycle
lw = 2

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
y_test=Y_test
y_score=Y_pred
n_classes=4
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(figsize=(10,8))
plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

# plt.plot(fpr["macro"], tpr["macro"],
#          label='macro-average ROC curve (area = {0:0.2f})'
#                ''.format(roc_auc["macro"]),
#          color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class CL- {0} (area = {1:0.2f})'
             ''.format(i+1, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()
plt.savefig("ROC_stack")

In [ ]:
%matplotlib inline
from keras.utils.vis_utils import plot_model
import matplotlib.image as mpimg

plot_model(model, to_file='LittleVGG.png', show_shapes=True, show_layer_names=True)
img = mpimg.imread('LittleVGG.png')
plt.figure(figsize=(100,70))
imgplot = plt.imshow(img) 

# Activation Maximization 

In [ ]:
from vis.visualization import visualize_activation
from vis.utils import utils
from keras import activations
from matplotlib import pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (18, 6)

# Utility to search for layer index by name. 
# Alternatively we can specify this as -1 since it corresponds to the last layer.
layer_idx = utils.find_layer_idx(model, 'mbm')

# Swap softmax with linear
model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)

# This is the output node we want to maximize.
filter_idx = 0
img = visualize_activation(model, layer_idx, filter_indices=filter_idx)
#img = visualize_activation(model, layer_idx, filter_indices=filter_idx, input_range=(0., 1.), verbose=True)
for output_idx in np.arange(5):
    img = visualize_activation(model, layer_idx, filter_indices=output_idx, input_range=(0., 1.))
    plt.figure()
    plt.title('Networks perception of OA level--{}'.format(output_idx))
    plt.imshow(img[..., 0])



## Visualizing Filter Patterns

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt

input_image_path = './OAI_Classification_Images/224*224/validation/level_1/  9001695 1.jpg'

# Show our input Image for Feature visualization
img1 = image.load_img(input_image_path)
plt.imshow(img1);
img_size = (224, 224)
# load imamge into a 4D Tensor, convert it to a numpy array and expand to 4 dim
img1 = image.load_img(input_image_path, target_size = img_size)
image_tensor = image.img_to_array(img1)
#print(image_tensor.shape)
image_tensor = image_tensor/255
image_tensor = np.expand_dims(image_tensor, axis=0)
#print(img.shape)

In [ ]:
from keras import models

# Extracts the top 8 layers
layer_outputs = [layer.output for layer in model.layers[:13]]

# Creates a model that returns these outputs given the model input
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

In [ ]:
activations = activation_model.predict(image_tensor)
first_layer_activation = activations[0]
print(first_layer_activation.shape)

In [ ]:
for i in range(0,32):
    plt.matshow(first_layer_activation[0, :, :,i], cmap='viridis')

In [ ]:
layer_names = []
for layer in model.layers[:13]:
    layer_names.append(layer.name)
images_per_row = 16

# Get CONV layers only
conv_layer_names = []
for layer_name in layer_names:
    if 'conv2d' in layer_name:
        conv_layer_names.append(layer_name)

for layer_name, layer_activation in zip(conv_layer_names, activations):
    n_features = layer_activation.shape[-1]
    size = layer_activation.shape[1]
    
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,:, :,col * images_per_row + row]
            
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
            row * size : (row + 1) * size] = channel_image
            
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
    scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')